# Convolutional Neural Network

### Part 1 - Building the CNN

#### Importing the Tensorflow libraries and packages

In [1]:
import sys

import tensorflow.keras
import pandas as pd
import sklearn as sk
import tensorflow as tf

print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tensorflow.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

Tensor Flow Version: 2.6.0
Keras Version: 2.6.0

Python 3.8.13 | packaged by conda-forge | (default, Mar 25 2022, 05:59:45) [MSC v.1929 64 bit (AMD64)]
Pandas 1.4.3
Scikit-Learn 1.1.1
GPU is available


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

import os

In [3]:
classifier = Sequential()

#classes = 2

#Kernels usually 1x1 | 3x3 | 5x5

#Adding a convolutional layer
classifier.add(Conv2D(8, (3, 3), input_shape = (150, 150, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

#Adding a second convolutional layer
classifier.add(Conv2D(16, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(0.30))
classifier.add(Conv2D(16, (3, 3), padding='same', activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))
#Flattening
classifier.add(Flatten())

#Full connection
classifier.add(Dense(units = 32, activation = 'relu'))
classifier.add(Dropout(0.30))
classifier.add(Dense(units = 1, activation= 'sigmoid'))

Let's double check the model description:

In [4]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 16)        0

In [5]:
from tensorflow.keras.optimizers import Adamax
opt = Adamax(learning_rate=0.001)
classifier.compile(optimizer = opt, loss = 'binary_crossentropy', metrics = ['accuracy'])

### Part 2 - Fitting the CNN to the images


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

path_dir = '../../dataset/train'
parth_dir_test = '../../dataset/test'

datagen = ImageDataGenerator(
                    rescale=1./255,
                    validation_split = 0.2)

train_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    shuffle=True,
    subset='training',
    batch_size = 64,
    seed = 11,
    class_mode = 'binary'
)
validation_generator = datagen.flow_from_directory(
    path_dir,
    target_size=(150,150),
    subset='validation',
    batch_size = 64,
    seed = 11,
    class_mode = 'binary'
)


Found 2202 images belonging to 2 classes.
Found 550 images belonging to 2 classes.


In [7]:
train_generator.class_indices

{'nonviolence': 0, 'violence': 1}

In [8]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard
import datetime

#rm -rf /logs/

es = EarlyStopping(monitor='val_loss', patience=25)
mc = ModelCheckpoint('best_cnn.h5', monitor='val_loss', mode='min', save_best_only=True)
reduceLR = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10)

# Create a TensorBoard callback
log_dir = "logs\\fit\\" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [9]:
classifier.fit(train_generator,
                         epochs = 200,
                         callbacks = [tensorboard_callback,es,mc,reduceLR],
                         validation_data = validation_generator
                         )

Epoch 1/200
35/35 [==============================] - 11s 237ms/step - loss: 0.7004 - accuracy: 0.5064 - val_loss: 0.6912 - val_accuracy: 0.5000
Epoch 2/200
35/35 [==============================] - 8s 233ms/step - loss: 0.6863 - accuracy: 0.5468 - val_loss: 0.6765 - val_accuracy: 0.5127
Epoch 3/200
35/35 [==============================] - 9s 245ms/step - loss: 0.6614 - accuracy: 0.6303 - val_loss: 0.6341 - val_accuracy: 0.7455
Epoch 4/200
35/35 [==============================] - 8s 228ms/step - loss: 0.6152 - accuracy: 0.6807 - val_loss: 0.5658 - val_accuracy: 0.7927
Epoch 5/200
35/35 [==============================] - 9s 248ms/step - loss: 0.5881 - accuracy: 0.6921 - val_loss: 0.5578 - val_accuracy: 0.7491
Epoch 6/200
35/35 [==============================] - 8s 225ms/step - loss: 0.5737 - accuracy: 0.7148 - val_loss: 0.5682 - val_accuracy: 0.7018
Epoch 7/200
35/35 [==============================] - 9s 248ms/step - loss: 0.5556 - accuracy: 0.7284 - val_loss: 0.5129 - val_accuracy: 0.789

Epoch 58/200
35/35 [==============================] - 8s 228ms/step - loss: 0.2922 - accuracy: 0.8833 - val_loss: 0.2787 - val_accuracy: 0.8818
Epoch 59/200
35/35 [==============================] - 8s 221ms/step - loss: 0.2817 - accuracy: 0.8833 - val_loss: 0.2899 - val_accuracy: 0.8709
Epoch 60/200
35/35 [==============================] - 8s 221ms/step - loss: 0.2755 - accuracy: 0.8901 - val_loss: 0.3193 - val_accuracy: 0.8545
Epoch 61/200
35/35 [==============================] - 8s 222ms/step - loss: 0.2871 - accuracy: 0.8896 - val_loss: 0.2626 - val_accuracy: 0.8855
Epoch 62/200
35/35 [==============================] - 8s 219ms/step - loss: 0.2715 - accuracy: 0.8996 - val_loss: 0.2660 - val_accuracy: 0.8909
Epoch 63/200
35/35 [==============================] - 8s 223ms/step - loss: 0.2687 - accuracy: 0.8919 - val_loss: 0.3058 - val_accuracy: 0.8564
Epoch 64/200
35/35 [==============================] - 8s 217ms/step - loss: 0.2712 - accuracy: 0.8978 - val_loss: 0.3198 - val_accuracy:

In [10]:
#Overfitting -> Your training accuracy will be higher than the accuracy on the validation/test set

#Overfitting indicates that your model is too complex for the problem that it is solving,
#filters in the case of Convolutional Neural Networks, and layers in the case of overall Deep Learning Models

#How do you know if your model is underfitting? Your model is underfitting if the accuracy on the validation set 
#is higher than the accuracy on the training set. Additionally, 
#if the whole model performs bad this is also called underfitting.

#https://towardsdatascience.com/deep-learning-3-more-on-cnns-handling-overfitting-2bd5d99abe5d

In [11]:
%tensorboard --logdir logs/fit

# Part 3 test the CNN

In [12]:
# Load Model
classifier.load_weights('best_cnn.h5')

In [13]:
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 150, 150, 8)       224       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 75, 75, 8)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 75, 75, 16)        1168      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 37, 37, 16)        0         
_________________________________________________________________
dropout (Dropout)            (None, 37, 37, 16)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 37, 37, 16)        2320      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 18, 18, 16)        0

In [14]:
test_set = datagen.flow_from_directory(parth_dir_test,
                                        target_size= (150,150),
                                        batch_size = 64,
                                        seed = 11,
                                        class_mode = 'binary',
                                        shuffle=False)

score = classifier.evaluate(test_set, verbose=0)

print('Test Loss', score[0])
print('Test accuracy', score[1])

Found 688 images belonging to 2 classes.
Test Loss 0.2876919209957123
Test accuracy 0.895348846912384


In [15]:
test_set.classes

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

In [16]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score

yhat_classes = classifier.predict_classes(test_set, verbose=0)
yhat_classes = yhat_classes[:, 0]

print(classification_report(test_set.classes,yhat_classes))

C:\Users\bruno\anaconda3\envs\gpu\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


              precision    recall  f1-score   support

           0       0.89      0.90      0.90       344
           1       0.90      0.89      0.89       344

    accuracy                           0.90       688
   macro avg       0.90      0.90      0.90       688
weighted avg       0.90      0.90      0.90       688

